# dMRI Preprocessing

This note book processes multiband diffusion weighted imaging for tensor-based analyses

In [ ]:
from nipype.pipeline.engine import Workflow, Node, JoinNode, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, FreeSurferSource
from nipype.interfaces.fsl import Reorient2Std, BET, MeanImage, MCFLIRT, FLIRT, ApplyTOPUP, TOPUP, Eddy, ExtractROI
from nipype.interfaces.freesurfer import MRIConvert, Binarize, FSCommand

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Study-specific variables
project_home = '/moochie/Cat/EmoGrow/dMRI_proc'
output_dir = project_home + '/proc/preprocessing'
workflow_dir = project_home + '/workflows'
raw_dir = '/moochie/R01-EmoGrow/MRI_Data'
phase_encoding_file = project_home + '/misc/acq_params.txt'
eddy_index_file = project_home + '/misc/eddy_index_params.txt'

#subjects_list = ['1000','1001','1002','1003','1004','1005','1006',
#                 '1007','1008','1009','1011','1012','1015','1016',
#                 '1017','1019','1021','1023','1025','1026', '1027','3000']
subjects_list = open(project_home + '/misc/emogrow_subjects.txt').read().splitlines()
#subjects_list = ['018']

# FreeSurfer set up - change SUBJECTS_DIR 
fs_dir = '/moochie/Cat/Aggregate_anats/subjects_dir'
FSCommand.set_default_subjects_dir(fs_dir)

In [ ]:
### Universal nodes

# get subjects list
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

# FreeSurferSource - Data grabber specific for FreeSurfer data
fssource = Node(FreeSurferSource(subjects_dir=fs_dir),
                run_without_submitting=True,
                name='fssource')

def FSid(subjid):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    
    fs_id = 'E'+str(subjid)
    return(fs_id)

fsid = Node(Function(input_names=['subjid'],
                     output_names=['fs_id'], 
                     function=FSid), 
            name='fsid')

# Sink data of interest
substitutions = [('_subjid_', '')] #output file name substitutions
datasink = Node(DataSink(base_directory = output_dir,
                        container = output_dir,
                        substitutions = substitutions), 
                name='datasink')

## Unwarping workflow

This workflow includes both unwarping and eddy correction.

In [ ]:
# Select func dicoms
def cvt_dicom(seqname,studypath,subjid):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    from glob import glob
    from nipype.interfaces.freesurfer import MRIConvert
    from os.path import abspath
    
    subjid = str(subjid)
    dicoms = glob('%s/%s/%s*/MR.*' % (studypath, subjid, seqname))
    print('%s/%s/%s*/MR.*' % (studypath, subjid, seqname))
    dicom_file=dicoms[0]
    
    cvt = MRIConvert()
    cvt.inputs.out_type = 'niigz'
    cvt.inputs.in_file = dicom_file
    cvt.inputs.out_file = seqname[0:13] + '.nii.gz'
    cvt.run()
    
    nii_file = abspath(seqname[0:13] + '.nii.gz')
    
    bvec = nii_file.replace('.nii.gz','.voxel_space.bvecs')
    bval = nii_file.replace('.nii.gz','.bvals')
    return(nii_file, bvec, bval)

def sort_pes(pes):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    
    from nipype.interfaces.fsl import Merge, ExtractROI
    from os.path import abspath
    
    eroi = ExtractROI()
    eroi.inputs.t_min = 0
    eroi.inputs.t_size = 1
    
    for i in [0,1]:
        if 'dMRI_dir98_AP' in pes[i]:
            pe0 = pes[i]
            pe_AP = pes[i]
        elif 'dMRI_dir98_PA' in pes[i]:
            pe_PA = pes[i]
    
    eroi.inputs.in_file = pe_AP
    eroi.inputs.roi_file = 'dMRI_AP.nii.gz' 
    eroi.run()
    pe_AP_1 = abspath('dMRI_AP.nii.gz')
    
    eroi.inputs.in_file = pe_PA
    eroi.inputs.roi_file = 'dMRI_PA.nii.gz' 
    eroi.run()
    pe_PA_1 = abspath('dMRI_PA.nii.gz')
        
    me = Merge()
    me.inputs.in_files = [pe_AP_1,pe_PA_1]
    me.inputs.dimension='t'
    me.inputs.merged_file = 'merged_pes.nii.gz'
    me.run()
    
    merged_pes = abspath('merged_pes.nii.gz')
    
    bvec = pe0.replace('.nii.gz','.voxel_space.bvecs')
    bval = pe0.replace('.nii.gz','.bvals')
    
    return(merged_pes, pe0, bvec, bval)

In [ ]:
# select all dMRI sequences
selectdmri=Node(Function(input_names=['seqname','studypath','subjid'],
                         output_names=['nii_file','bvec','bval'], 
                         function=cvt_dicom),
                name='selectdmri')
selectdmri.inputs.studypath = raw_dir
selectdmri.iterables = ('seqname',['dMRI_dir98_AP_14','dMRI_dir98_PA_14'])

# Sort the files for unwarping
sort_pe_list = JoinNode(Function(input_names=['pes'],
                                 output_names=['merged_pes','pe0','bvec','bval'],
                                 function=sort_pes), 
                        name='sort_pe_list', joinsource='selectdmri', joinfield='pes')

# actually do the unwarping
topup = Node(TOPUP(encoding_file=phase_encoding_file), name='topup')

apply_topup = Node(ApplyTOPUP(in_index=[1], encoding_file=phase_encoding_file,
                              method='jac', out_corrected='dmri_unwarped.nii.gz'),
                   name='apply_topup')

# reorient unwarped image to MNI space standard
reorientdmri = Node(Reorient2Std(terminal_output='file'),
                    name='reorientdmri')

# average dMRI so we can make a mask
avg_dmri = Node(MeanImage(), name='avg_dmri')

# create a dMRI space brain mask using BET
make_mask = Node(BET(mask=True), name='make_mask')

# Eddy correction
eddy = Node(Eddy(use_cuda=False, in_index=eddy_index_file, 
                 in_acqp=phase_encoding_file), name='eddy')

In [ ]:
prepreprocflow = Workflow(name='prepreprocflow')
prepreprocflow.connect([(infosource,selectdmri, [('subjid','subjid')]),
                        (selectdmri,sort_pe_list, [('nii_file','pes'),
                                                   ('bvec','bvecs'),
                                                   ('bval','bvals')]),
                        (sort_pe_list,topup, [('merged_pes','in_file')]),
                        (topup, apply_topup, [('out_fieldcoef','in_topup_fieldcoef'), 
                                              ('out_movpar','in_topup_movpar')]),
                        (sort_pe_list, apply_topup, [('pe0','in_files')]),
                        (apply_topup, reorientdmri, [('out_corrected','in_file')]),
                        (reorientdmri, avg_dmri, [('out_file','in_file')]),
                        (avg_dmri, make_mask, [('out_file','in_file')]),
                        (make_mask, datasink, [('mask_file','in_mask')]),
                        #(reorientdmri, eddy, [('out_file','in_file')]),
                        (sort_pe_list, datasink, [('bvec','in_bvec'),
                                              ('bval','in_bval')]),
                        #(topup, eddy, [('out_movpar','in_topup_movpar'),
                        #               ('out_fieldcoef','in_topup_fieldcoef')]),
                        #(eddy, datasink, [('out_rotated_bvecs','rotated_bvecs'),
                        #                  ('out_parameter','field_and_movement_params'),
                        #                  ('out_corrected','eddy_corrected_dmri'),
                        #                  ('out_restricted_movement_rms','dmri_motion')]),
                        (reorientdmri, datasink, [('out_file','unwarped_funcs')])
                       ])

prepreprocflow.base_dir = workflow_dir
prepreprocflow.write_graph(graph2use='flat')
prepreprocflow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

## Preprocessing Workflow

This workflow performs the following steps:
* coregistration between dMRI and T1w anat
* registration to sample-specific template

In [ ]:
def isolate_ToM(dtk_nii):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from numpy import zeros_like
    from os.path import abspath
    
    # r precuneus, l precuneus, r supramarginal, l supramarginal, r IPL, l IPL, l rostral-anterior cing, r rostral-anterior cing, l mofc, r mofc
    labels = [2025,1025,2031,1031,2008,1008,2026,1026,2014,1014] 
    
    aparc_nii = load(dtk_nii)
    aparc_data = aparc_nii.get_data()
    
    ToM_data = zeros_like(aseg_data) 
    for x in labels:
        ToM_data[aparc_data == x] = x
    ToM_nifti = Nifti1Image(ToM_data, aparc_nii.affine)
    save(ToM_nifti, 'ToM_rois.nii.gz')
    ToM_file = abspath('ToM_rois.nii.gz')
    
    return(ToM_file)

In [ ]:
## Nodes for creating ROI volumes from freesurfer segmentation
file_template = {'dMRI': output_dir + '/unwarped_funcs/{subjid}/dmri_unwarped_reoriented.nii.gz'}
selectfiles = Node(SelectFiles(file_template),name='selectfiles')

# trim to B0 volume
trim_b0 = Node(ExtractROI(t_min=0, t_size=1), name='trim_b0')

# convert anat to nifti
t1_to_nii = Node(MRIConvert(out_file='T1w.nii.gz',out_type='niigz'),
                 name='t1_to_nii')
# reorient anat
reorient_t1 = Node(Reorient2Std(terminal_output='file'),
                    name='reorient_t1')

#convert parc to nifti
parc_to_nii = Node(MRIConvert(out_file='aparcDTK.nii.gz',out_type='niigz'),
                   name='parc_to_nii')

#reorient parc
reorient_parc = Node(Reorient2Std(terminal_output='file'),
                     name='reorient_parc')

# register anat to B0
coreg_t1_b0 = Node(FLIRT(out_file='warp_t1.nii.gz',out_matrix_file='xfm.mat'), name='coreg_t1_b0')

# apply registration to DK parcellation
apply_reg = Node(FLIRT(out_file='warp_dtk.nii.gz',apply_xfm=True), name='apply_reg')

# strip down to mentalizing network ROIs
create_ToM_ROIs = Node(Function(input_names= ['dtk_nii'], output_names=['ToM_file'],name=isolate_ToM), 
                       name='create_ToM_ROIs')


In [ ]:
roiprocflow = Workflow(name='roiprocflow')
roiprocflow.connect([(infosource, fsid, [('subjid','subjid')]), 
                     (fsid, fssource, [('fs_id','subject_id')]), 
                     (fssource, t1_to_nii, [('brain', 'in_file')]),
                     (t1_to_nii, reorient_t1, [('out_file','in_file')]),
                     (reorient_t1, coreg_t1_b0, [('out_file','in_file')]),
                     
                     (fssource, parc_to_nii, [('aparc_aseg', 'in_file')]),
                     (parc_to_nii, reorient_parc, [('out_file','in_file')]),
                     (reorient_parc, apply_reg, [('out_file','in_file')]),
                     (coreg_t1_b0, apply_reg, [('out_matrix_file','in_matrix_file')]),
                     
                     (infosource, selectfiles, [('subjid','subjid')]),
                     (selectfiles, trim_b0, [('dMRI','in_file')]),
                     (trim_b0, coreg_t1_b0, [('roi_file','reference')]),
                     (trim_b0, apply_reg, [('roi_file','reference')]),
                     (apply_reg, create_ToM_ROIs, [('out_file','dtk_nii')]),
                     
                     (apply_reg, datasink, [('out_file','warped_DTK')]),
                     (coreg_t1_b0, datasink, [('out_file','warped_T1')]),
                     (trim_b0, datasink, [('roi_file','B0_vol_files')]),
                     (create_ToM_ROIs, [('ToM_file','ToM_ROIs')])
                    ])
roiprocflow.base_dir = workflow_dir
roiprocflow.write_graph(graph2use='flat')
roiprocflow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})